In [29]:
from groq import Groq
import os
from dotenv import load_dotenv
import requests

load_dotenv()
client=Groq(api_key=os.environ['GROQ'])

ModuleNotFoundError: No module named 'llama_index'

In [2]:
messages = [
    ChatMessage(
        role="system", content:"""
        You are a medical entity recognition system. Your task is to extract information and return a JSON object with exactly three keys: "amount", "expenditure", and "status". 

1. Extract the monetary value and assign it to the "amount" key.
2. Identify the category of expenditure and assign it to the "expenditure" key. The expenditure should be classified into one of the following categories: "tests", "doctor", or "medicine".
3. If the input is related to medical expenses, set the "status" key to True. If the input is unrelated or outside your scope, set the "status" key to False.

Ensure the output is a simple JSON object with no additional text or nested structures.

"""
    ),
    ChatMessage(role="user", content="spent 1k rs on neuro surgeon"),
]
resp = llm.chat(messages)

data = resp.model_dump()  # This gives you the full response as a dictionary
message_content = data['message']['content']  # Extract the 'content' part

NameError: name 'ChatMessage' is not defined

In [31]:
chat_completion = client.chat.completions.create(
    messages=[
         {"role":"system", "content":"""
        You are a medical entity recognition system. Your task is to extract information and return a JSON object with exactly three keys: "amount", "expenditure", and "status". 

        1. Extract the monetary value and assign it to the "amount" key and value should be whole number.
        2. Identify the category of expenditure and assign it to the "expenditure" key. The expenditure should be classified into one of the following categories: "tests", "doctor", or "medicine".
        3. If the input is related to medical expenses, set the "status" key to "true". If the input is unrelated or outside your scope, set the "status" key to "false".
        
        Ensure the output is a simple JSON object with no additional text or nested structures.
        give me the json with no preamble and the value of status key will be in double quotes cumpolsurily
        """
         },
         {
            "role": "user",
            "content": "spent 13k rs on neuro surgeon",
        }
    ],
    model="llama3-70b-8192",
)

resp=chat_completion.choices[0].message.content

In [32]:
eval(resp)

{'amount': 13000, 'expenditure': 'doctor', 'status': 'true'}

In [1]:
import psycopg2
import pandas as pd
from sqlalchemy import create_engine,text as to_sql_text


db_username = 'myuser'
db_password = 'root'
db_host = 'localhost'
db_port = '5432'
db_name = 'medwell_db'

connection_string = f'postgresql+psycopg2://{db_username}:{db_password}@{db_host}:{db_port}/{db_name}'
engine = create_engine(connection_string)
conn=engine.connect()

In [2]:
import pandas as pd

In [18]:
df=pd.read_sql_query('''select expense_type,amount,date(date) from patient_expense order by date desc limit 10;''',engine)

In [19]:
df['date'] = pd.to_datetime(df['date'])
df['date'] = df['date'].dt.strftime('%Y-%m-%d')

In [20]:
df.to_dict("records")

[{'expense_type': 'reports', 'amount': '500', 'date': '2024-10-26'},
 {'expense_type': 'doctor', 'amount': '10000', 'date': '2024-10-26'},
 {'expense_type': 'doctor', 'amount': '10000', 'date': '2024-10-26'}]

In [15]:
df

,expense_type,amount,date
0,reports,500,2024-10-26
1,doctor,10000,2024-10-26
2,doctor,10000,2024-10-26


In [21]:
df=pd.read_sql_query('''
SELECT expense_type,SUM(CAST(amount AS DECIMAL)) as total
from patient_expense
where user_id=43
group by expense_type
;
''',engine)

In [28]:
df.to_dict("list")

{'expense_type': ['doctor', 'reports'], 'total': [20000.0, 500.0]}

In [29]:
df=pd.read_sql_query('''
SELECT CAST(amount AS DECIMAL) as expenses
from patient_expense
where user_id=43

;
''',engine)

In [31]:
df.to_dict("list")

{'expenses': [500.0, 10000.0, 10000.0]}

In [36]:
df=pd.read_sql_query('''
SELECT to_char(date, 'Month') AS month_name,sum(CAST(amount AS DECIMAL)) as expenses,date_part('month', date) AS month
from patient_expense
where user_id=43
group by to_char(date, 'Month'),date_part('month', date)
order by date_part('month', date)
;
''',engine)[["month_name","expenses"]]

In [37]:
df.to_dict("list")

{'month_name': ['October  '], 'expenses': [20500.0]}

In [38]:
df=pd.read_sql_query('''
SELECT sum(CAST(amount AS DECIMAL)) as expenses,date_part('year', date) AS year
from patient_expense
where user_id=43
group by date_part('year', date)
order by date_part('year', date)
;
''',engine)
df.to_dict("list")

{'expenses': [20500.0], 'year': [2024.0]}

In [39]:
df=pd.read_sql_query('''
SELECT date_part('month', date) AS month,expense_type,sum(CAST(amount AS DECIMAL)) as expenses
from patient_expense
where user_id=43
group by date_part('month', date),expense_type
order by date_part('month', date)
;
;
''',engine)
df.to_dict("list")

{'month': [10.0, 10.0],
 'expense_type': ['doctor', 'reports'],
 'expenses': [20000.0, 500.0]}

In [48]:
df.to_dict("list")

{'month': [10.0, 10.0],
 'expense_type': ['doctor', 'reports'],
 'expenses': [20000.0, 500.0]}

In [61]:
grouped_df = df.groupby(by="month")

# Convert to the desired JSON-like structure
result = []

In [62]:
for month,df in grouped_df:
    print(month)
    print(df)
    month_data = {
        "month": month,
        "data": df[["expense_type","expenses"]].to_dict("list")
    }
    result.append(month_data)

10.0
   month expense_type  expenses
0   10.0       doctor   20000.0
1   10.0      reports     500.0


In [63]:
result

[{'month': 10.0,
  'data': {'expense_type': ['doctor', 'reports'],
   'expenses': [20000.0, 500.0]}}]

In [ ]:
SELECT date_part('year', date) AS month,expense_type,sum(CAST(amount AS DECIMAL)) as expenses
from patient_expense
where user_id=43
group by date_part('year', date),expense_type
order by date_part('year', date)
;

In [81]:
df=pd.read_sql_query('''
    select date_of_report,report_type,report_data from patient_report r join patient_reportdetail d on r.id=d.report_id 
    order by r.submitted_at desc
    limit 5;
    ''',engine)

In [22]:
def generate_report_text(report_data):
    report_text = ""
    for test, details in report_data.items():
        min_val = details['min']
        max_val = details['max']
        unit = details['unit']
        value = details['value']
        if value==-1:continue
        
        value_text = f"{value} {unit}" if value != -1 else "Not available"
        
        report_text += f"{test.capitalize()}: Normal range {min_val}-{max_val} {unit}, Current value: {value_text}\n"
    
    return report_text

In [82]:
df["report_data"]=df.report_data.apply(generate_report_text)

In [14]:
df.iloc[0].report_data

'Pcv: Normal range 36-50 %, Current value: 32 %\nIron: Normal range 60-170 μg/dL, Current value: Not available\nSodium: Normal range 135-145 mmol/L, Current value: Not available\nAlbumin: Normal range 3.5-5.0 g/dL, Current value: Not available\nCalcium: Normal range 8.5-10.2 mg/dL, Current value: Not available\nChloride: Normal range 98-107 mmol/L, Current value: Not available\nGlobulin: Normal range 2.0-4.0 g/dL, Current value: Not available\nProteins: Normal range 6.0-8.0 g/dL, Current value: Not available\nPotassium: Normal range 3.5-5.0 mmol/L, Current value: Not available\nRbc_count: Normal range 4.2-5.4 million cells/μL, Current value: 3.98 million cells/μL\nWbc_count: Normal range 4.5-11.0 thousand cells/μL, Current value: 5100 thousand cells/μL\nBlood_urea: Normal range 7-20 mg/dL, Current value: Not available\nHemoglobin: Normal range 12.1-15.5 g/dL, Current value: 10.8 g/dL\nPhosphorus: Normal range 2.5-4.5 mg/dL, Current value: Not available\nLymphocytes: Normal range 20-45 

In [83]:
df

,date_of_report,report_type,report_data
0,11/09/22,blood_report,"Pcv: Normal range 36-50 %, Current value: 32 %..."
1,3/8/24,blood_report,"Pcv: Normal range 36-50 %, Current value: 38 %..."
2,13/8/24,blood_report,"Pcv: Normal range 36-50 %, Current value: 31 %..."


In [84]:
entire_data_text=""

In [85]:
for index,data in df.iterrows():
    text=f'''
    report date:{data.date_of_report} report type : {data.report_type}
    report data :
    {data.report_data}
    \n
    '''
    entire_data_text+=text
    

In [86]:
len(entire_data_text)

2826

In [28]:
len(entire_data_text)

2826

In [70]:
from langchain_core.output_parsers import JsonOutputParser
parser=JsonOutputParser()

In [87]:
chat_completion = client.chat.completions.create(
    messages=[
         {"role":"system", "content":"""
        You will receive a user's complete medical history, including:

Date of report
Type of report
Detailed report data with various metrics, each indicating its normal range, unit, and current measured value.
Task: Analyze this data to:

Provide an overall health summary of approximately 400 words that:

Highlights trends in critical health metrics and any notable fluctuations over time.
Identifies deficiencies, excesses, or patterns that may indicate potential health concerns 
Assesses the user’s overall health status, detailing how well their results align with healthy ranges and discussing implications of any recurring deviations from these ranges.
Create a personalized diet and nutrition plan that:

Recommends specific foods and dietary adjustments to address the user’s unique needs (e.g., foods rich in iron, vitamin D, or low in sodium).
Includes general advice to support balanced nutrition and ongoing health improvement based on observed trends.
Output Format: Return a JSON object structured as follows, with no extra text or preamble:

"summary": A string containing the detailed health summary.
"plans": A string containing the personalized diet and nutrition plan.
Return the JSON in a precise format with no additional whitespace before or after keys and no preamble.
        """
         },
         {
            "role": "user",
            "content": entire_data_text,
        }
    ],
    model="llama3-70b-8192",
)

resp=chat_completion.choices[0].message.content

In [68]:
eval(resp)["summary"]

SyntaxError: unterminated string literal (detected at line 3) (<string>, line 3)

In [88]:
print(parser.parse(resp)["plans"])

Based on the user's unique needs, a personalized diet and nutrition plan is recommended. To address anemia, the user should increase consumption of iron-rich foods such as red meat, spinach, and fortified cereals. Vitamin C-rich foods like citrus fruits, bell peppers, and broccoli can help increase iron absorption. To support balanced nutrition, the user should increase consumption of fruits, vegetables, whole grains, and lean proteins. 

Additionally, the user should limit sodium intake to address potential kidney dysfunction. Foods rich in vitamin D, calcium, and phosphorus can help support bone health. Foods high in antioxidants like berries, nuts, and leafy greens can help reduce inflammation. 

It is essential to consult with a healthcare professional to develop a comprehensive treatment plan that addresses the user's specific health needs.


In [20]:
df=pd.read_sql_query(sql=to_sql_text('''select round(avg(cast(hemoglobin as decimal)),2) as avg_hemoglobin,
    round(avg(cast(rbc_count as decimal)),2) as avg_rbc_count,
    round(avg(cast(wbc_count as decimal)),2) as avg_wbc_count
    from patient_report p join patient_reportdetail r on r.report_id=p.id
    where p.user_id=43'''),con=conn)

In [18]:

df["submitted_at"]=df.submitted_at.astype(str)

In [24]:
df.to_dict("records")[0]

{'avg_hemoglobin': 11.48, 'avg_rbc_count': 4.32, 'avg_wbc_count': 8075.0}